In [ ]:
!pip install -q 'git+https://github.com/drscook/voting_predictor'

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1 MB 4.8 MB/s 
     |████████████████████████████████| 16.6 MB 185 kB/s 
     |████████████████████████████████| 7.8 MB 68.5 MB/s 
     |████████████████████████████████| 1.6 MB 57.0 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 132 kB 63.0 MB/s 


In [1]:
%reload_ext google.colab.data_table
%reload_ext autotime
%reload_ext autoreload
%autoreload 2

root_path = '/content/drive/'
local = False
local = True
if local:
    import sys, google.colab
    google.colab.drive.mount(root_path)
    sys.path.insert(0, root_path + 'MyDrive/helpers')
    sys.path.insert(0, root_path + 'MyDrive/voting_predictor')
    root_path += 'MyDrive/gerrymandering/2022-12/'
else:
    !pip install -q --force-reinstall --no-deps 'git+https://github.com/drscook/helpers' 'git+https://github.com/drscook/voting_predictor'

from helpers.common_imports import *
from helpers import utilities as ut
import voting_predictor as vp

self = vp.Redistricter(
    census_api_key = 'dccb7bb4b7df5dff59d2d99c859016f973197e4e',
    bg_project_id = 'redistricting-361203',
    state = 'RI',
    root_path = root_path,
)

# self.get_crosswalks()#overwrite=True)
# self.get_assignments()#overwrite=True)
# self.get_shapes()#overwrite=True)
# self.get_pl()#overwrite=True)
self.combine(overwrite=True)
# vp.test.f(5)
# ut.

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
assignments.RI2020 ... Elapsed time: 0.1093 seconds
shapes.RI2020 ... Elapsed time: 0.0684 seconds
pl.RI2020 ... Elapsed time: 0.0878 seconds
combine.RI2020 fetching ... 
select
    *
from
    assignments.RI2020 as A
inner join
    shapes.RI2020 as S
using
    (block2020)
--on
  --  A.block2020 = S.block2020
inner join
    pl.RI2020 as P
using
    (block2020)
    --A.block2020 = P.block2020

Elapsed time: 4.9553 seconds


'combine.RI2020'